In [10]:
import pyodbc
import csv
import json
import base64
from cryptography.fernet import Fernet
import pandas as pd
import numpy as np
from sqlalchemy import create_engine



cipher_suite = Fernet(b'l0cD3IJGNVc4c6dArV_u8XkXMffeumPFNd4hltjoiA8=')

# Gibt den entschlüsselten Zugang zur ERP DB zurück
def load_credential(file_path):
    with open(file_path, 'rb') as file:
        encrypted_data = file.read()
        
    decrypted_data = cipher_suite.decrypt(encrypted_data)
    credentials = json.loads(decrypted_data.decode('utf8'))
    
    return credentials['username'], credentials['password']

def load_query(file_path):
    try:
        with open(file_path, 'r') as file:
            query = file.read()
        return query
    except FileNotFoundError as file_error:
        print("SQL-Datei: ",file_error)

try:

    #key = b'wDByOLLImijDsEaiGL1gzGFsXruC_7ol1Ht5lqo='
    #cipher_suite = Fernet(key)

    #
    # Einlesen der verschlüsselten Zugangsdaten.
    #
    credential_file = "C:\\Temp\\fet_test_credentials.enc"
    username, password = load_credential(credential_file)

   #
   # Angabe des Servers und der Datenbank
    #
    server = 'PFDusSQL,50432'
    database = 'fet_test'

    connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL+Server'
    sql_query = load_query(r'\\V-Processes\\ERP_Queries\\SKU_Carton_4_QS.sql')

    engine = create_engine(connection_string)
    with engine.connect() as conn:
        df = pd.read_sql_query(sql_query, conn)
        csv_filename = r'\\pfduskommi\Promodoro.Export\Warehouse\SKU_Carton_4_QS.csv'
        df['Gewicht_kg'] = df['Gewicht_kg'].astype(str)
        df.to_csv(csv_filename, sep=';', index=False, encoding='utf-8-sig')

except Exception as e:
    print(e)
    x = input()

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Kontraktnummer           984 non-null    object        
 1   Sendungsnummer           984 non-null    object        
 2   Wareneingang             984 non-null    object        
 3   Bestellnummer            984 non-null    object        
 4   CREATION_DATE            984 non-null    datetime64[ns]
 5   Kartonbezeichnung        984 non-null    object        
 6   Kartonnummer             984 non-null    object        
 7   Gewicht_kg               984 non-null    object        
 8   StÃ¼ck                   984 non-null    int64         
 9   Artikelnummer            984 non-null    object        
 10  Lieferantenname          984 non-null    object        
 11  Lieferantnummer          984 non-null    object        
 12  Artikel_gesamt           984 non-nul

In [9]:
df.head()

,Kontraktnummer,Sendungsnummer,Wareneingang,Bestellnummer,CREATION_DATE,Kartonbezeichnung,Kartonnummer,Gewicht_kg,StÃ¼ck,Artikelnummer,Lieferantenname,Lieferantnummer,Artikel_gesamt,Status,VorgangBestellungImport
0,KT2300000053,SP-2025/0001,WE-000170,BE00000611,2025-01-03 15:29:10.983,5099-steel gray-XS - 1,1,8.8,20,5099-steel gray-XS,Metro Knitting & Dyeing Mills Ltd.,00025,20,1000.0,aef67290-105d-11d9-4c85-9d915831e9eb
1,KT2300000053,SP-2025/0001,WE-000170,BE00000611,2025-01-03 15:29:10.983,5099-steel gray-S - 2,2,4.6,10,5099-steel gray-S,Metro Knitting & Dyeing Mills Ltd.,00025,10,1000.0,aef67290-105d-11d9-4c85-9d915831e9eb
2,KT2300000053,SP-2025/0001,WE-000170,BE00000611,2025-01-03 15:29:10.983,5099-steel gray-4XL - 3,3,40.0,50,5099-steel gray-4XL,Metro Knitting & Dyeing Mills Ltd.,00025,50,1000.0,aef67290-105d-11d9-4c85-9d915831e9eb
3,KT2400000001,SP-1-2024-000021,WE-1-2025-000186,BE-1-2025-000655,2025-05-28 11:47:47.820,00001-graphite-M-1,1,11.33,25,2199-graphite-M,Metro Knitting & Dyeing Mills Ltd.,00025,400,1000.0,aef67290-105d-11d9-4c85-9d915831e9eb
4,KT2400000001,SP-1-2024-000021,WE-1-2025-000186,BE-1-2025-000655,2025-05-28 11:47:47.820,00001-graphite-M-2,2,11.33,25,2199-graphite-M,Metro Knitting & Dyeing Mills Ltd.,00025,400,1000.0,aef67290-105d-11d9-4c85-9d915831e9eb
